In [1]:
# !git clone https://github.com/clovaai/CRAFT-pytorch.git

In [2]:
import os
import cv2
import easyocr
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

In [3]:
def read_coordinates_from_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file if line.strip()]

def crop_text_regions(image_path, coordinates):
    image = Image.open(image_path)
    
    cropped_images = []
    for i, coord in enumerate(coordinates):
        coord = [int(x) for x in coord.split(',')]
        
        left = min(coord[0], coord[6])
        top = min(coord[1], coord[3])
        right = max(coord[2], coord[4])
        bottom = max(coord[5], coord[7])
        
        cropped = image.crop((left, top, right, bottom))
        cropped_images.append(cropped)
    
    return cropped_images

In [4]:
processor = TrOCRProcessor.from_pretrained("team-lucid/trocr-small-korean")
model = VisionEncoderDecoderModel.from_pretrained("team-lucid/trocr-small-korean")

In [5]:
image_path = "/home/pervinco/upstage-cv-classification-cv7/dataset/train/0a4adccbb7fe73e0.jpg"
coordinates_file_path = "/home/pervinco/upstage-cv-classification-cv7/dataset/train_with_bbox/res_0a4adccbb7fe73e0.txt"

coordinates = read_coordinates_from_file(coordinates_file_path)
cropped_images = crop_text_regions(image_path, coordinates)

In [6]:
extracted_texts = []
for i, cropped_image in enumerate(cropped_images):
    pixel_values = processor(cropped_image, return_tensors="pt").pixel_values
    
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # print(f"텍스트 영역 {i + 1}: {generated_text}")
    # plt.figure()
    # plt.imshow(cropped_image)
    # plt.show()
    
    extracted_texts.append(generated_text)

combined_text = " ".join(extracted_texts)

search_word = "임신"
if search_word in combined_text:
    print(f"'{search_word}'이(가) 추출된 텍스트에 포함되어 있습니다.")
else:
    print(f"'{search_word}'이(가) 추출된 텍스트에 포함되어 있지 않습니다.")

print(f"결합된 텍스트: {combined_text}")

/home/pervinco/miniconda3/envs/cls-project/lib/python3.9/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'임신'이(가) 추출된 텍스트에 포함되어 있습니다.
결합된 텍스트: 환자등록번호 건강보험 임신 진료비 지원 신청 및 임신서 출산 주민등록번호 수진지 《잉부》 전화번호 자 택 휴대전화: 이헤일 요양기관 확인란[ 2016 임신확인 2016 뿐만여정일 다태아구분 □ 삼 태 아 일 태 아 □ 다 태 이 □ 쌍 태아 이상 위에 기록한 사창이 사실임을  확인함 2016 03 년 31 요양문화 미즈메디병원 < 1 1 1 담당의사[면허] 국민건강보험법 시행령 의하여 신청합니다 제 23조에 임신 출산 진료 2016 03 년 31 1 신청인 <세영 또는  인) 전화번호 수진자(잉신부)와의  관계 국민건강보험공단  이사장  귀하 구비세류 수진파【임신부】와의 관계를 입 중 할 수 있는 서류[가족애 신참한 김우이 반 중1 ' 주민등록표등본 가족관계증명서


In [7]:
# def augment_text_regions(image_path, coordinate_path, output_path, num_augmentations=1):
#     image = cv2.imread(image_path)
#     coords = read_coordinates_from_file(coordinate_path)

#     for i in range(num_augmentations):
#         # 랜덤한 회색 배경 생성 (흰색 - 회색 범위)
#         background_color = np.random.randint(128, 256)  # 128(회색)에서 255(흰색) 사이의 랜덤 값
#         background = np.full(image.shape, background_color, dtype=np.uint8)
        
#         for coord in coords:
#             x1, y1, x2, y2, x3, y3, x4, y4 = map(int, coord.split(','))
            
#             # 다각형 마스크 생성
#             pts = np.array([[x1,y1], [x2,y2], [x3,y3], [x4,y4]], np.int32)
#             pts = pts.reshape((-1,1,2))
#             mask = np.zeros(image.shape[:2], np.uint8)
#             cv2.fillPoly(mask, [pts], (255))

#             # 마스크를 사용하여 원본 이미지에서 텍스트 영역 추출
#             text_region = cv2.bitwise_and(image, image, mask=mask)

#             # 배경에 텍스트 영역 붙여넣기
#             background = cv2.add(background, text_region)

#         output_file = os.path.join(output_path, f"augmented_{i}.png")
#         cv2.imwrite(output_file, background)

In [8]:
# augment_text_regions(image_path, coordinates_file_path, "./", num_augmentations=5)

In [9]:
# def augment_text_regions(image_path, coordinate_path, output_path, num_augmentations=1):
#     image = cv2.imread(image_path)
#     coords = read_coordinates_from_file(coordinate_path)

#     for i in range(num_augmentations):
#         avg_color = np.mean(image, axis=(0, 1)).astype(int)
#         background_color = avg_color + np.random.randint(-20, 21, 3) 
#         background_color = np.clip(background_color, 0, 255)
        
#         h, w = image.shape[:2]
#         background = np.zeros((h, w, 3), dtype=np.uint8)
#         for c in range(3):
#             background[:, :, c] = np.linspace(background_color[c] - 30, background_color[c] + 30, w)
        
#         mask = np.zeros(image.shape[:2], dtype=np.uint8)
#         for coord in coords:
#             x1, y1, x2, y2, x3, y3, x4, y4 = map(int, coord.split(','))
            
#             pts = np.array([[x1,y1], [x2,y2], [x3,y3], [x4,y4]], np.int32)
#             pts = pts.reshape((-1,1,2))
#             cv2.fillPoly(mask, [pts], (255))

#         blurred_mask = cv2.GaussianBlur(mask, (21, 21), 0)
#         blurred_mask = blurred_mask.reshape(h, w, 1) / 255.0

#         text_regions = image * blurred_mask
#         background = background * (1 - blurred_mask)
#         result = text_regions + background

#         output_file = os.path.join(output_path, f"augmented_{i}.png")
#         cv2.imwrite(output_file, result.astype(np.uint8))

In [10]:
# augment_text_regions(image_path, coordinates_file_path, "./", num_augmentations=5)